In [1]:
!pip install -q PyAthena

## Prepare Datasets for Predictor Training, Validation and Testing

In [1]:
from dataproc.cohort import query_esbl_pts, remove_dups, observation_window
from dataproc.sampling import generate_samples
from hyper_params import HyperParams
import pandas as pd

In [2]:
# load hyperparams instance
params = HyperParams()

In [3]:
# Select esbl microbiology test
esbl_admits = query_esbl_pts()
# Remove dups
esbl_admits = remove_dups(esbl_admits)
# Create observation window
esbl_admits_window = observation_window(esbl_admits, window_size=params.observation_window_hours)
# Subset columns
pts_labels = esbl_admits_window[['hadm_id', 'RESISTANT_YN']]
pts_labels.to_pickle('data/patient_labels.pkl')

In [4]:
pts_labels = pd.read_pickle('data/patient_labels.pkl')
pts_labels.head()

,hadm_id,RESISTANT_YN
4262,101757,0
4267,186474,0
4268,194730,0
4269,112086,0
4270,158569,1


In [5]:
# Loading the features
from dataproc.create_dataset import dataset_creation

features = dataset_creation(pts_labels['hadm_id'])
features.to_pickle('data/features.pkl')

In [6]:
features = pd.read_pickle('data/features.pkl')
features.head()

,hadm_id,subject_id,10378-8,10535-3,10839-9,11555-0,11556-8,11557-6,11558-4,13362-9,...,9322-9,admittime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity,gender
0,100021,29971,NaN,NaN,NaN,3,90,51,7.38,RANDOM,...,NaN,2109-08-17 10:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicaid,SPAN,UNOBTAINABLE,MARRIED,HISPANIC OR LATINO,M
1,100045,1569,2+,NaN,NaN,0,152,35,7.44,RANDOM,...,NaN,2176-02-05 18:40:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,F
2,100050,29633,NaN,NaN,NaN,0,519,40,7.40,RANDOM,...,NaN,2179-09-19 13:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,M
3,100053,24912,NaN,NaN,NaN,-8,119,30,7.33,RANDOM,...,NaN,2124-07-14 03:19:00,EMERGENCY,EMERGENCY ROOM ADMIT,Private,NaN,NOT SPECIFIED,SINGLE,WHITE,M
4,100113,58240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2153-01-05 00:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,Medicare,ENGL,JEWISH,SINGLE,WHITE,M


In [7]:
loinc_codes = list(features.drop(columns=['hadm_id', 'subject_id', 'admittime']).columns)[:-8]

In [8]:
# Embedding the features
from dataproc.embeddings import loinc_values

loinc_vals = loinc_values(loinc_codes)
loinc_vals.dropna(subset=['value'], inplace=True)
loinc_vals = loinc_vals.astype({'value': 'string', 'loinc_code': 'category'})
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.lstrip('LESS THAN '))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.lstrip('GREATER THAN '))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.lstrip('>GREATER THAN '))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.lstrip('<LESS THAN '))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.rstrip(' NG/ML'))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.lstrip('<>'))
loinc_vals['value'] = loinc_vals['value'].map(lambda x: x.replace(',', '.'))
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'UNABLE TO ANALYZE'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'MOLYSIS FALSELY DECREASES THIS RESULT'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'COMPUTER NETWORK FAILURE. TEST NOT RESULTED.'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'UNABLE TO DETERMINE'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == ':UNABLE TO DETERMINE'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'UNABLE TO QUANTITATE'].index),  inplace=True)
loinc_vals.drop(list(loinc_vals.loc[loinc_vals['value'] == 'UNABLE TO REPORT'].index),  inplace=True)

In [9]:
numeric = []
categorical = []
weird = []
for code in loinc_codes:
    size = len(loinc_vals.loc[loinc_vals['loinc_code'] == str(code), 'value'])
    size_unique = len(loinc_vals.loc[loinc_vals['loinc_code'] == str(code), 'value'].unique())
    sum_na = pd.to_numeric(loinc_vals.loc[loinc_vals['loinc_code'] == str(code), 'value'], errors='coerce').isna().sum()
    if sum_na / size < 0.05:
        numeric.append(code)
    elif sum_na / size > 0.05 and size_unique < 100:
        categorical.append(code)
    else:
        weird.append(code)

In [10]:
len(loinc_codes)

144

In [11]:
len(numeric)

95

In [12]:
len(categorical)

38

In [13]:
len(weird)

11

In [14]:
weird

['3297-9',
 '4023-8',
 '4548-4',
 '48065-7',
 '5642-4',
 '5787-7',
 '5793-5',
 '5796-8',
 '5808-1',
 '5821-4',
 '6773-6']

In [15]:
worth_it = ['6773-6','5808-1','5787-7','5821-4']

In [18]:
numeric_stats = []
for code in numeric:
    a = pd.to_numeric(loinc_vals.loc[loinc_vals['loinc_code'] == str(code), 'value'], errors='coerce').describe()
    numeric_stats.append(a)
numeric_stats_df = pd.concat(numeric_stats, axis=1, keys=numeric)

In [19]:
numeric_stats_df

,10535-3,10839-9,11555-0,11556-8,11557-6,11558-4,1644-4,1742-6,1751-7,17849-1,...,763-3,772-4,777-3,785-6,786-4,787-2,788-0,789-8,804-5,9322-9
count,8358.000000,11447.000000,490527.000000,490523.000000,490504.000000,530658.000000,24986.000000,219448.000000,146690.000000,10073.000000,...,75587.000000,16564.000000,778241.000000,747560.000000,747921.000000,747537.000000,746408.000000,747568.000000,753127.000000,18128.000000
mean,1.129834,7.479856,-0.090604,136.737019,42.736738,7.379366,167.710318,129.960002,3.178038,2.534686,...,3.046492,13.766542,239.295564,30.220467,33.508648,90.264973,15.799282,3.510249,10.499787,3.920383
std,0.749934,16.189266,5.353903,92.114782,11.388981,0.087115,229.124373,536.810586,0.757973,1.775861,...,6.148193,59.786263,150.321123,2.574024,1.573992,6.987865,2.356128,0.656843,9.006713,2.365156
min,0.020000,0.000000,-414.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.900000,0.000000,...,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000
25%,0.600000,0.300000,-3.000000,81.000000,36.000000,7.330000,85.000000,18.000000,2.600000,1.400000,...,0.000000,1.000000,139.000000,28.900000,32.500000,86.000000,14.100000,3.050000,6.400000,2.800000
50%,1.000000,0.400000,0.000000,109.000000,41.000000,7.390000,124.000000,32.000000,3.100000,2.100000,...,0.000000,3.000000,215.000000,30.300000,33.600000,90.000000,15.300000,3.440000,9.100000,3.500000
75%,1.400000,4.700000,3.000000,156.000000,47.000000,7.440000,187.000000,69.000000,3.700000,3.200000,...,3.000000,8.000000,307.000000,31.600000,34.600000,94.000000,17.000000,3.900000,12.800000,4.500000
max,8.000000,575.000000,162.000000,1914.000000,247.000000,7.990000,12496.000000,25460.000000,6.900000,22.100000,...,94.000000,2879.000000,4504.000000,48.900000,45.800000,147.000000,36.400000,32.970000,846.700000,65.400000


In [18]:
for code in worth_it:
    print(len(loinc_vals.loc[loinc_vals['loinc_code'] == str(code), 'value'].unique()))

663
843
108
734


In [6]:
# splot to train / validation / test sets
df_train, df_validation, df_test = generate_samples(df_dataset = pts_labels,
                 negative_to_positive_ratio = params.negative_to_positive_ratio,
                 test_set_fraction = params.test_set_fraction,
                 validation_set_fraction = params.validation_set_fraction,
                 random_state = params.random_state)

In [14]:
# persist datasets
df_train.to_csv('data/df_predictor_train.csv')
df_validation.to_csv('data/df_predictor_validation.csv')
df_test.to_csv('data/df_predictor_test.csv')

## Prepare Datasets for Encoder Training

In [1]:
from dataproc.cohort import query_esbl_pts, remove_dups, observation_window
from dataproc.sampling import generate_samples
from hyper_params import HyperParams
import pandas as pd

In [8]:
# load hyperparams instance
params = HyperParams()

In [ ]:
# TODO James
# TODO Run the query that loads all patient data into => df_all_patients_data
assert df_all_patients_data

In [6]:
# splot to train / validation / test sets
df_embedding_train, df_embedding_validation, df_embedding_test = generate_samples(df_dataset = df_all_patients_data,
                 negative_to_positive_ratio = params.negative_to_positive_ratio,
                 test_set_fraction = params.test_set_fraction,
                 validation_set_fraction = params.validation_set_fraction,
                 random_state = params.random_state)

In [7]:
df_embedding_train.to_csv('data/df_embedding_train.csv')
df_embedding_validation.to_csv('data/df_embedding_validation.csv')
df_embedding_test.to_csv('data/df_embedding_test.csv')